In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import glob

# Import Data

In [16]:
data_files = [x for x in glob.glob('../data/clean/monthly/*.csv') if '001_250' in x]
rawdata_df = pd.concat(tqdm((pd.read_csv(x) for x in data_files), total = len(data_files)))
rawdata_df.sample(10)

,date,ticker,mod_time_m,time_m,price,datetime
347897,2019-10-25,DTE,157,13:09:47.593,127.930,2019-10-25 13:10:00
157787,2019-12-12,CFG,119,09:59:59.152,39.900,2019-12-12 10:00:00
67202,2019-07-05,XRAY,170,14:14:49.699,57.815,2019-07-05 14:15:00
175154,2020-05-14,AVB,158,13:14:45.413,150.040,2020-05-14 13:15:00
342689,2019-02-28,FBHS,177,14:49:55.826,46.930,2019-02-28 14:50:00
338084,2020-05-27,FTV,148,12:24:56.704,63.190,2020-05-27 12:25:00
147380,2019-04-11,AAP,160,13:24:59.859,179.240,2019-04-11 13:25:00
259237,2019-03-21,AMP,177,14:49:10.318,130.050,2019-03-21 14:50:00
94209,2019-06-10,AFL,185,15:29:57.022,54.320,2019-06-10 15:30:00
290946,2019-09-24,DOW,132,11:04:55.505,47.140,2019-09-24 11:05:00


# Clean Data

In [17]:
# Copy data for cleaning
data_df = rawdata_df.copy()

In [18]:
# Fix dates
data_df['date'] = pd.to_datetime(data_df['date'])
data_df['datetime'] = pd.to_datetime(data_df['datetime'])
data_df['time'] = data_df['datetime'].dt.time
data_df = data_df.drop(['time_m', 'mod_time_m'], axis = 1)
data_df = data_df.sort_values(by = ['ticker', 'date', 'datetime'])

In [19]:
# Adjust stock splits
data_df.loc[(data_df["ticker"] == "AAPL") & (data_df["datetime"] < "2020-08-31"), "price"] = (
    data_df.loc[(data_df["ticker"] == "AAPL") & (data_df["datetime"] < "2020-08-31"), "price"] / 5
)

In [20]:
%%time
# Prices
data_df['log_price'] = np.log(data_df['price'])
data_df['return'] = data_df.groupby(['ticker', 'date'])['log_price'].transform(pd.Series.diff)

CPU times: user 36 s, sys: 3.28 s, total: 39.3 s
Wall time: 39.5 s


In [21]:
%%time
# Return vars
data_df["return_p2"] = np.power(data_df["return"], 2)
data_df["return_p4"] = np.power(data_df["return"], 4)
data_df["return_lag1"] = data_df.groupby(["ticker", "date"])["return"].shift(1)
data_df["return_bp"] = np.abs(np.multiply(data_df["return"], data_df["return_lag1"]))

# BV
data_df["bipower_var_daily"] = data_df.groupby(["ticker", "date"])["return_bp"].transform("sum") * (
    78 / 77
)
data_df["bipower_var_tod"] = data_df.groupby(["ticker", "time"])["return_bp"].transform("mean")

# Jump detection
alpha = 4
data_df["tod_correction"] = np.divide(
    data_df["bipower_var_tod"], data_df.groupby(["ticker"])["bipower_var_tod"].transform("mean")
)
data_df["jump_cut"] = (
    alpha * np.sqrt(data_df["bipower_var_daily"] * data_df["tod_correction"]) * ((1 / 78) ** (0.49))
)
data_df['is_jump'] = np.abs(data_df['return']) > data_df['jump_cut']

## Filtered diffusive returns
# Main
data_df['return_d'] = data_df['return']*(1-data_df['is_jump'])
data_df['return_d_pos'] = data_df['return_d']*(data_df['return_d'] > 0)
data_df['return_d_neg'] = data_df['return_d']*(data_df['return_d'] < 0)
# Semi
data_df['return_d_p2'] = np.power(data_df['return_d'], 2)
data_df['return_d_p4'] = np.power(data_df['return_d'], 4)
data_df['return_d_p2_pos'] = np.power(data_df['return_d_pos'], 2)
data_df['return_d_p4_pos'] = np.power(data_df['return_d_pos'], 4)
data_df['return_d_p2_neg'] = np.power(data_df['return_d_neg'], 2)
data_df['return_d_p4_neg'] = np.power(data_df['return_d_neg'], 4)

## Filtered jump returns
# Main
data_df['return_j'] = data_df['return']*(data_df['is_jump'])
data_df['return_j_pos'] = data_df['return_j']*(data_df['return_j'] > 0)
data_df['return_j_neg'] = data_df['return_j']*(data_df['return_j'] < 0)
# Semi
data_df['return_j_p2'] = np.power(data_df['return_j'], 2)
data_df['return_j_p4'] = np.power(data_df['return_j'], 4)
data_df['return_j_p2_pos'] = np.power(data_df['return_j_pos'], 2)
data_df['return_j_p4_pos'] = np.power(data_df['return_j_pos'], 4)
data_df['return_j_p2_neg'] = np.power(data_df['return_j_neg'], 2)
data_df['return_j_p4_neg'] = np.power(data_df['return_j_neg'], 4)

data_df.head()

CPU times: user 9.86 s, sys: 10.9 s, total: 20.8 s
Wall time: 11.1 s


,date,ticker,price,datetime,time,log_price,return,return_p2,return_p4,return_lag1,...,return_d_p4_neg,return_j,return_j_pos,return_j_neg,return_j_p2,return_j_p4,return_j_p2_pos,return_j_p4_pos,return_j_p2_neg,return_j_p4_neg
0,2019-01-02,A,66.56,2019-01-02 09:35:00,09:35:00,4.198104,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-02,A,66.24,2019-01-02 09:40:00,09:40:00,4.193285,-0.004819,0.000023,5.394249e-10,NaN,...,5.394249e-10,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-01-02,A,66.31,2019-01-02 09:45:00,09:45:00,4.194341,0.001056,0.000001,1.244496e-12,-0.004819,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-01-02,A,66.38,2019-01-02 09:50:00,09:50:00,4.195396,0.001055,0.000001,1.239252e-12,0.001056,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-01-02,A,66.28,2019-01-02 09:55:00,09:55:00,4.193888,-0.001508,0.000002,5.166071e-12,0.001055,...,5.166071e-12,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# TOD check
#data_df.query('ticker == "AAPL"').plot(x = 'time', y = 'tod_correction')

In [23]:
# Daily sums
columns_return = [x for x in data_df.columns if 'return' in x]
data_daily_df = data_df.groupby(["ticker", "date"])[columns_return].sum().reset_index()
data_daily_df

,ticker,date,return,return_p2,return_p4,return_lag1,return_bp,return_d,return_d_pos,return_d_neg,...,return_d_p4_neg,return_j,return_j_pos,return_j_neg,return_j_p2,return_j_p4,return_j_p2_pos,return_j_p4_pos,return_j_p2_neg,return_j_p4_neg
0,A,2019-01-02,-0.013157,0.000225,2.333611e-09,-0.014681,0.000146,-0.013157,0.045040,-0.058197,...,1.800605e-09,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
1,A,2019-01-03,-0.030796,0.000670,2.167020e-08,-0.029217,0.000459,-0.030796,0.074811,-0.105607,...,1.282101e-08,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
2,A,2019-01-04,0.016799,0.000247,2.141852e-09,0.016799,0.000181,0.016799,0.064591,-0.047792,...,6.078688e-10,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
3,A,2019-01-07,0.012791,0.000171,1.819701e-09,0.015031,0.000097,0.022322,0.048325,-0.026002,...,1.348027e-10,-0.009532,0.000000,-0.009532,0.000045,1.037729e-09,0.000000,0.000000e+00,0.000045,1.037729e-09
4,A,2019-01-08,0.000442,0.000219,3.155356e-09,0.000590,0.000143,-0.004315,0.041866,-0.046181,...,2.049808e-09,0.004758,0.004758,0.000000,0.000023,5.123606e-10,0.000023,5.123606e-10,0.000000,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95049,XRAY,2020-07-27,0.011799,0.000306,1.061692e-08,0.010878,0.000187,0.011799,0.052926,-0.041127,...,6.555731e-09,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
95050,XRAY,2020-07-28,-0.000230,0.000087,4.167782e-10,0.002411,0.000061,-0.003662,0.026332,-0.029994,...,1.949781e-10,0.003432,0.003432,0.000000,0.000012,1.387530e-10,0.000012,1.387530e-10,0.000000,0.000000e+00
95051,XRAY,2020-07-29,0.022128,0.000321,7.310644e-09,0.021458,0.000202,0.022128,0.067373,-0.045246,...,1.147863e-09,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
95052,XRAY,2020-07-30,0.026214,0.000268,4.969285e-09,0.025329,0.000148,0.026214,0.064260,-0.038046,...,9.802597e-10,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00


# Export Data

In [24]:
%%time
data_df.to_parquet('../data/processed/sp500_returns.parquet')
data_daily_df.to_parquet('../data/processed/sp500_returns_daily.parquet')

CPU times: user 10.4 s, sys: 1.42 s, total: 11.9 s
Wall time: 19.1 s
